In [2]:
import time
import torch
import cv2
import numpy as np
from tqdm import tqdm
# import skvideo
# skvideo.setFFmpegPath(r'C:\ProgramData\Anaconda3\Lib\site-packages\skvideo\io')
from torch.backends import cudnn
from pycocotools.coco import COCO
from backbone import EfficientDetBackbone
from efficientdet.utils import BBoxTransform, ClipBoxes
from utils.utils import preprocess, invert_affine, postprocess, preprocess_video
import skvideo.measure

# Defining thresholds for evaluation

In [2]:
# Taking the video file as input
video_src = r'C:\Users\Acer\Downloads\Untitled_video_720p.mp4'  

compound_coef = 2
force_input_size = None  

threshold = 0.5
iou_threshold = 0.5

use_cuda = True
use_float16 = False
cudnn.fastest = True
cudnn.benchmark = True

In [3]:
obj_list = ['background','3m''abus','accenture','adidas','adidas1','adidas_text','airhawk','airness','aldi','aldi_text','alfaromeo','allett','allianz','allianz_text',
            'aluratek','aluratek_text','amazon','amcrest','amcrest_text','americanexpress','americanexpress_text','android','anz','anz_text',
            'apc','apecase','apple','aquapac_text','aral','armani','armitron','aspirin','asus','at_and_t','athalon','audi','audi_text',
            'axa','bacardi','bankofamerica','bankofamerica_text','barbie','barclays','base','basf','batman','bayer','bbc','bbva',
          'becks','bellataylor','bellodigital','bellodigital_text','bem','benrus','bershka','bfgoodrich','bik','bionade',
            'blackmores','blizzardentertainment','bmw','boeing','boeing_text','bosch','bosch_text','bottegaveneta','bridgestone','bridgestone_text','budweiser','budweiser_text','bulgari','burgerking',
            'burgerking_text','calvinklein','canon','carglass','carlsberg','carters','cartier','caterpillar','chanel','chanel_text','cheetos','chevrolet',
            'chevrolet_text','chevron','chickfila','chimay','chiquita','cisco','citi','citroen','citroen_text','coach','cocacola','coke','colgate','comedycentral','converse','corona','corona_text',
            'costa','costco','cpa_australia','cvs','cvspharmacy','danone','dexia','dhl','disney','doritos','drpepper','dunkindonuts','ebay','ec','erdinger','espn','esso','esso_text',
            'evernote','facebook','fedex','ferrari','firefox','firelli','fly_emirates','ford','fosters','fritolay','fritos',
            'gap','generalelectric','gildan','gillette','goodyear','google','gucci','guinness','hanes','head','head_text','heineken','heineken_text','heraldsun','hermes','hersheys',
            'hh','hisense','hm','homedepot','homedepot_text','honda','honda_text','hp','hsbc','hsbc_text','huawei','huawei_text','hyundai','hyundai_text','ibm','ikea','infiniti''infiniti_text',
            'intel','internetexplorer','jackinthebox','jacobscreek','jagermeister','jcrew','jello','johnnywalker','jurlique','kelloggs',
            'kfc','kia','kitkat','kodak','kraft','lacoste','lacoste_text','lamborghini','lays','lego','levis','lexus','lexus_text',
            'lg','londonunderground','loreal','lotto','luxottica','lv','marlboro','marlboro_fig','marlboro_text','maserati','mastercard','maxwellhouse',
            'maxxis','mccafe','mcdonalds','mcdonalds_text','medibank','mercedesbenz','mercedesbenz_text','michelin','microsoft','milka',
            'millerhighlife','mini','miraclewhip','mitsubishi','mk','mobil','motorola','mtv','nasa','nb','nbc','nescafe','netflix','nike',
            'nike_text','nintendo','nissan','nissan_text','nivea','northface','nvidia','obey','olympics','opel','optus','optus_yes','oracle','pampers','panasonic'
            ,'paulaner','pepsi','pepsi_text','pepsi_text1','philadelphia','philips','pizzahut','pizzahut_hut','planters','playstation',
            'poloralphlauren','porsche','porsche_text','prada','puma','puma_text','quick','rbc','recycling','redbull','redbull_text','reebok','reebok1','reebok_text',
            'reeses','renault','republican','rittersport','rolex','rolex_text','ruffles','samsung','santander','santander_text','sap','schwinn',
            'scion_text','sega','select','shell','shell_text','shell_text1','siemens','singha','skechers','sony','soundcloud','soundrop',
            'spar','spar_text','spiderman','sprite','standard_liege','starbucks','stellaartois','subaru','subway','sunchips',
            'superman','supreme','suzuki','t-mobile','tacobell','target','target_text','teslamotors','texaco','thomsonreuters',
            'tigerwash','timberland','tissot','tnt','tommyhilfiger','tostitos','total','toyota','toyota_text','tsingtao','twitter','umbro',
            'underarmour','unicef','uniqlo','uniqlo1','unitednations','ups','us_president','vaio','velveeta','venus','verizon',
            'verizon_text','visa','vodafone','volkswagen','volkswagen_text','volvo','walmart','walmart_text','warnerbros','wellsfargo',
            'wellsfargo_text','wii','williamhill','windows','wordpress','xbox','yahoo','yamaha','yonex','yonex_text','youtube','zara']

# Predicting on input video frames
* Reading the video frames as and feeding them forward through model.
* Postprocessing the output, putting bounding boxes and writing it to the result video

In [ ]:
input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536]
input_size = input_sizes[2] 
output= r'C:\Users\Acer\Downloads\openlogo\vid\\'
# load model
model = EfficientDetBackbone(compound_coef=compound_coef, num_classes=len(obj_list))
model.load_state_dict(torch.load(f'weights/efficientdet-d2_4_9025.pth'))
model.requires_grad_(False)
model.eval()

if use_cuda:
    model = model.cuda()
if use_float16:
    model = model.half()

# function for writing on the frame
def display(preds, imgs):
    for i in range(len(imgs)):
        if len(preds[i]['rois']) == 0:
            return imgs[i]

        for j in range(len(preds[i]['rois'])):
            (x1, y1, x2, y2) = preds[i]['rois'][j].astype(np.int)
            obj = obj_list[preds[i]['class_ids'][j]]
            score = float(preds[i]['scores'][j])

            cv2.putText(imgs[i], '{}'.format(obj),
                        (x1, y1 + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        (0, 0, 255), 1)
        
        return imgs[i]
# Box
regressBoxes = BBoxTransform()
clipBoxes = ClipBoxes()

# Video capture
cap = cv2.VideoCapture(video_src)
fps = cap.get(cv2.CAP_PROP_FPS)

video = cv2.VideoWriter('result.mp4',
                        cv2.VideoWriter_fourcc(*'MP4V'),
                        int(fps), 
                        (720, 1280))  
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # frame preprocessing
    ori_imgs, framed_imgs, framed_metas = preprocess_video(frame, max_size=input_size)

    if use_cuda:
        x = torch.stack([torch.from_numpy(fi).cuda() for fi in framed_imgs], 0)
    else:
        x = torch.stack([torch.from_numpy(fi) for fi in framed_imgs], 0)

    x = x.to(torch.float32 if not use_float16 else torch.float16).permute(0, 3, 1, 2)

    # model predict
    with torch.no_grad():
        features, regression, classification, anchors = model(x)

        out = postprocess(x,
                        anchors, regression, classification,
                        regressBoxes, clipBoxes,
                        threshold, iou_threshold)

    # proceess ouput frame
    out = invert_affine(framed_metas, out)
    img_show = display(out, ori_imgs)
    # write video frame by frame
    video.write(img_show)
cap.release()
video.release()
